In [34]:
pip install datasets

                                              0.0/1.6 MB ? eta -:--:--
     ----------                               0.4/1.6 MB 8.9 MB/s eta 0:00:01
     --------------------------------         1.3/1.6 MB 13.8 MB/s eta 0:00:01
     ---------------------------------------- 1.6/1.6 MB 14.5 MB/s eta 0:00:00
                                              0.0/262.3 kB ? eta -:--:--
     ---------------------------------------- 262.3/262.3 kB ? eta 0:00:00
  Using cached mako-1.3.10-py3-none-any.whl (78 kB)
                                              0.0/226.5 kB ? eta -:--:--
     ------------------------------------- 226.5/226.5 kB 13.5 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.1.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


                                              0.0/515.2 kB ? eta -:--:--
     ------------------------------------  512.0/515.2 kB 16.2 MB/s eta 0:00:01
     ------------------------------------- 515.2/515.2 kB 10.7 MB/s eta 0:00:00
                                              0.0/27.5 MB ? eta -:--:--
     -                                        1.3/27.5 MB 42.6 MB/s eta 0:00:01
     ---                                      2.6/27.5 MB 33.3 MB/s eta 0:00:01
     -----                                    3.8/27.5 MB 30.3 MB/s eta 0:00:01
     -------                                  4.9/27.5 MB 28.6 MB/s eta 0:00:01
     ---------                                6.3/27.5 MB 28.8 MB/s eta 0:00:01
     -----------                              8.0/27.5 MB 30.3 MB/s eta 0:00:01
     ------------                             8.9/27.5 MB 28.6 MB/s eta 0:00:01
     --------------                          10.5/27.5 MB 28.5 MB/s eta 0:00:01
     -----------------                       12.5/27.5 


[notice] A new release of pip is available: 23.1.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [36]:
# import pandas as pd
# import numpy as np
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.optim import Adam
# from torch.utils.data import TensorDataset, DataLoader
# from gensim.models import Word2Vec
# from seqeval.metrics import classification_report, f1_score
# from tqdm import tqdm
# import os

from gensim.models import Word2Vec
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score, recall_score, classification_report
import transformers
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler, TensorDataset
from transformers import TrainingArguments, AutoTokenizer, EvalPrediction, AutoModelForTokenClassification, Trainer
import sys
import os
import argparse
import torch
import ast
# from datasets import load_dataset, ClassLabel, Value, Sequence, load_from_disk
from tqdm import tqdm
import time
from datasets import load_dataset, ClassLabel, Value, Sequence, load_from_disk


### `CNN Classification`

In [29]:
SEQUENCE_LENGTH = 128
# EMBEDDING_DIM = 100
# BATCH_SIZE = 32
# EPOCHS = 100
# LEARNING_RATE = 0.001
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# bert_model = args.model
MAX_LEN = 100
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 4
EPOCHS = 100
LEARNING_RATE = 0.001
bert_model = "camembert-base"
nb_labels = 2
tokenizer = AutoTokenizer.from_pretrained(bert_model)

In [ ]:
W2V_PATH = "w2v_med_cbow.model"

TRAIN_FILES = [
    "TP_ISD2020/QUAERO_FrenchMed/MEDLINE/MEDLINEtrain_layer1_ID.csv",
    "TP_ISD2020/QUAERO_FrenchMed/EMEA/EMEAtrain_layer1_ID.csv",
]
VALID_FILES = [
    "TP_ISD2020/QUAERO_FrenchMed/MEDLINE/MEDLINEdev_layer1_ID.csv",
    "TP_ISD2020/QUAERO_FrenchMed/EMEA/EMEAdev_layer1_ID.csv",
]
TEST_FILES = [
    "TP_ISD2020/QUAERO_FrenchMed/MEDLINE/MEDLINEtest_layer1_ID.csv",
    "TP_ISD2020/QUAERO_FrenchMed/EMEA/EMEAtest_layer1_ID.csv",
]

In [8]:
# --- 2. ROBUST DATA LOADER ---
def load_data_from_csv(file_paths):
    all_sentences = []
    all_tags = []

    for fpath in file_paths:
        if not os.path.exists(fpath):
            print(f"❌ File not found: {fpath}")
            continue
        print(f"Loading {os.path.basename(fpath)}...", end=" ")

        try:
            # key: skip_blank_lines=False preserves sentence separators
            df = pd.read_csv(
                fpath,
                sep=None,
                engine="python",
                keep_default_na=False,
                skip_blank_lines=False,
            )
        except:
            print("Read Failed.")
            continue

        # Detect columns
        if "Mot" in df.columns and "Tag" in df.columns:
            words, tags = df["Mot"].astype(str).values, df["Tag"].astype(str).values
        else:
            words, tags = (
                df.iloc[:, 0].astype(str).values,
                df.iloc[:, -1].astype(str).values,
            )

        curr_s, curr_t = [], []
        file_s, file_t = [], []

        for w, t in zip(words, tags):
            if not w.strip():  # Empty line = Sentence Break
                if curr_s:
                    file_s.append(curr_s)
                    file_t.append(curr_t)
                    curr_s, curr_t = [], []
            else:
                curr_s.append(w)
                curr_t.append(t)
        if curr_s:
            file_s.append(curr_s)
            file_t.append(curr_t)

        # Fallback for giant files (Chunking)
        if len(file_s) < 10 and len(words) > 500:
            flat_w = [w for s in file_s for w in s]
            flat_t = [t for s in file_t for t in s]
            file_s = [
                flat_w[i : i + SEQUENCE_LENGTH]
                for i in range(0, len(flat_w), SEQUENCE_LENGTH)
            ]
            file_t = [
                flat_t[i : i + SEQUENCE_LENGTH]
                for i in range(0, len(flat_t), SEQUENCE_LENGTH)
            ]

        print(f"-> {len(file_s)} sentences.")
        all_sentences.extend(file_s)
        all_tags.extend(file_t)

    return all_sentences, all_tags


print("--- LOADING DATA ---")
train_sents, train_tags = load_data_from_csv(TRAIN_FILES)
valid_sents, valid_tags = load_data_from_csv(VALID_FILES)
test_sents, test_tags = load_data_from_csv(TEST_FILES)

--- LOADING DATA ---
Loading MEDLINEtrain_layer1_ID.csv... -> 91 sentences.
Loading EMEAtrain_layer1_ID.csv... -> 120 sentences.
Loading MEDLINEdev_layer1_ID.csv... -> 90 sentences.
Loading EMEAdev_layer1_ID.csv... -> 106 sentences.
Loading MEDLINEtest_layer1_ID.csv... -> 94 sentences.
Loading EMEAtest_layer1_ID.csv... -> 97 sentences.


In [14]:
# --- 3. BUILD VOCAB & INITIALIZE EMBEDDINGS (TP1) ---
print("\n--- INITIALIZING EMBEDDINGS ---")

# A. Build Vocabulary from Training Data
word_counts = {}
for sent in train_sents:
    for word in sent:
        word_counts[word] = word_counts.get(word, 0) + 1

vocab = sorted(word_counts, key=word_counts.get, reverse=True)
word2idx = {w: i + 2 for i, w in enumerate(vocab)}  # Start at 2
word2idx["<PAD>"] = 0
word2idx["<UNK>"] = 1
idx2word = {v: k for k, v in word2idx.items()}

# B. Load Word2Vec and Create Weight Matrix
w2v_model = Word2Vec.load(W2V_PATH)
vocab_size = len(word2idx)
embedding_matrix = np.zeros((vocab_size, MAX_LEN))
hits = 0

for word, idx in word2idx.items():
    if word in w2v_model.wv:
        embedding_matrix[idx] = w2v_model.wv[word]
        hits += 1
    elif word.lower() in w2v_model.wv:  # Case Insensitive Fallback
        embedding_matrix[idx] = w2v_model.wv[word.lower()]
        hits += 1
    else:
        # Initialize OOV words with random noise
        embedding_matrix[idx] = np.random.normal(scale=0.6, size=(MAX_LEN,))

print(f"Vocab Size: {vocab_size}")
print(f"Pre-trained Weights Found: {hits}/{vocab_size} ({hits/vocab_size:.1%})")

# Convert to FloatTensor
embedding_weights = torch.tensor(embedding_matrix, dtype=torch.float32).to(DEVICE)


--- INITIALIZING EMBEDDINGS ---
Vocab Size: 5775
Pre-trained Weights Found: 5615/5775 (97.2%)


In [20]:

# --- 4. DATA PREPARATION (ENCODING) ---
def encode_sequences(seqs, mapping, default_val, max_len=128):
    encoded = []
    for s in seqs:
        seq = [mapping.get(item, default_val) for item in s]
        if len(seq) < max_len:
            seq += [0] * (max_len - len(seq))
        else:
            seq = seq[:max_len]
        encoded.append(seq)
    return torch.tensor(encoded, dtype=torch.long)


# Encode Words (Input)
X_train = encode_sequences(train_sents, word2idx, 1)  # 1 = UNK
X_valid = encode_sequences(valid_sents, word2idx, 1)
X_test = encode_sequences(test_sents, word2idx, 1)

# Encode Tags (Target)
tag_set = set(t for s in train_tags + valid_tags + test_tags for t in s)
tag2idx = {t: i + 1 for i, t in enumerate(sorted(list(tag_set)))}
tag2idx["<PAD>"] = 0
idx2tag = {v: k for k, v in tag2idx.items()}
print(f"Tags: {tag2idx}")

y_train = encode_sequences(train_tags, tag2idx, 0)  # 0 = PAD (ignored in loss)
y_valid = encode_sequences(valid_tags, tag2idx, 0)
y_test = encode_sequences(test_tags, tag2idx, 0)

# Loaders
train_loader = DataLoader(
    TensorDataset(X_train, y_train), shuffle=True, batch_size=TRAIN_BATCH_SIZE
)
valid_loader = DataLoader(
    TensorDataset(X_valid, y_valid), shuffle=False, batch_size=VALID_BATCH_SIZE
)
test_loader = DataLoader(
    TensorDataset(X_test, y_test), shuffle=False, batch_size=VALID_BATCH_SIZE
)

Tags: {'B-ANAT': 1, 'B-CHEM': 2, 'B-DEVI': 3, 'B-DISO': 4, 'B-GEOG': 5, 'B-LIVB': 6, 'B-OBJC': 7, 'B-PHEN': 8, 'B-PHYS': 9, 'B-PROC': 10, 'I-ANAT': 11, 'I-CHEM': 12, 'I-DEVI': 13, 'I-DISO': 14, 'I-GEOG': 15, 'I-LIVB': 16, 'I-OBJC': 17, 'I-PHEN': 18, 'I-PHYS': 19, 'I-PROC': 20, 'O': 21, '<PAD>': 0}


In [25]:
model = AutoModelForTokenClassification.from_pretrained(bert_model, num_labels=nb_labels)
for name, param in model.named_parameters():
    print(name, param.requires_grad)

training_args = TrainingArguments(
    f"finetune_" + bert_model.replace("/","_") + "_" + str(EPOCHS) + "_" + str(TRAIN_BATCH_SIZE),
    save_strategy = "epoch",
    do_eval=True,
    eval_strategy="epoch",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=TRAIN_BATCH_SIZE,
    num_train_epochs=EPOCHS,
    load_best_model_at_end=True)


Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


roberta.embeddings.word_embeddings.weight True
roberta.embeddings.position_embeddings.weight True
roberta.embeddings.token_type_embeddings.weight True
roberta.embeddings.LayerNorm.weight True
roberta.embeddings.LayerNorm.bias True
roberta.encoder.layer.0.attention.self.query.weight True
roberta.encoder.layer.0.attention.self.query.bias True
roberta.encoder.layer.0.attention.self.key.weight True
roberta.encoder.layer.0.attention.self.key.bias True
roberta.encoder.layer.0.attention.self.value.weight True
roberta.encoder.layer.0.attention.self.value.bias True
roberta.encoder.layer.0.attention.output.dense.weight True
roberta.encoder.layer.0.attention.output.dense.bias True
roberta.encoder.layer.0.attention.output.LayerNorm.weight True
roberta.encoder.layer.0.attention.output.LayerNorm.bias True
roberta.encoder.layer.0.intermediate.dense.weight True
roberta.encoder.layer.0.intermediate.dense.bias True
roberta.encoder.layer.0.output.dense.weight True
roberta.encoder.layer.0.output.dense.bia

In [61]:
# helper functions for dataset preprocessing
def string_list_2_list(x):
    if type(x["Tag"][0]) == list:
        x["Tag"] = list(map(ast.literal_eval,x["Tag"]))
    return x

def encode(examples):
#    print (examples) 
    r = tokenizer(list(map(str, examples['review'])), padding=True, truncation=True, max_length=MAX_LEN)
    #r = tokenizer(list(map(str, examples['text'])), padding=True, truncation=True, max_length=MAX_LEN)
    return r

def preprocess_labels(examples):
    if type(examples['Tag'][0]) == list:
        examples['Tag'] = [[float(x) for x in label] for label in examples['Tag']]

    else:
        examples['Tag'] = [float(x) for x in examples['Tag']]
    return examples





def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    f1_macro_average = f1_score(y_true=y_true, y_pred=y_pred, average='macro')
    precision_weighted_average = precision_score(y_true=y_true, y_pred=y_pred, average='weighted')
    recall_average = recall_score(y_true=y_true, y_pred=y_pred, average='weighted')
    f1_weighted_average = 2.0 * ((precision_weighted_average * recall_average) / ( precision_weighted_average + recall_average ))
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    print(classification_report(y_pred, y_true))
    # return as dictionary
    metrics = {'precision weighted': precision_weighted_average,
               'recall weighted': recall_average,
               'f1 weighted': f1_weighted_average,
               'accuracy': accuracy}
    return metrics

def multi_label_metrics_v2(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    return (classification_report(y_true, y_pred))

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    result = multi_label_metrics(predictions=preds,labels=p.label_ids)
    return result

def padding(examples):
    for key in ['input_ids', 'attention_mask']:
        for i, element in enumerate(examples[key]):
            if len(element) < 20:
                for i in range(20 - len(element)):
                    element.append(0)
            else:
                examples[key][i] = element[:20]
    return examples

def preprocess_dataset(csvfile, binfile, kind='train'):
    l_data = load_dataset('csv', data_files={kind: [csvfile]})
    #data = load_dataset(csvfile, split=kind)
    l_data = l_data.map(string_list_2_list, batched=True)
    l_data = l_data.map(preprocess_labels, batched=True)
    l_data = l_data.map(encode, batched=True)
    new_features = l_data[kind].features.copy()
    print(new_features)
    if type(l_data[kind]["Tag"][0]) == list:
        new_features["Tag"] = Sequence(feature=Value(dtype='float'))
    else:
        new_features["Tag"] = feature=Value(dtype='float')
    print(new_features)
    l_data = l_data[kind].cast(new_features)
    l_data = l_data.map(padding, batched=True)
    l_data.save_to_disk(binfile)


In [62]:
train_binary = 'TP_ISD2020/QUAERO_FrenchMed/MEDLINE/MEDLINEtrain_layer1_ID.csv' + "_" + bert_model.replace("/","_") + "_" + str(MAX_LEN) + ".bin"
valid_binary = 'TP_ISD2020/QUAERO_FrenchMed/MEDLINE/MEDLINEdev_layer1_ID.csv' + "_" + bert_model.replace("/","_") + "_" + str(MAX_LEN) + ".bin"
test_binary = 'TP_ISD2020/QUAERO_FrenchMed/MEDLINE/MEDLINEtest_layer1_ID.csv' + "_" + bert_model.replace("/","_") + "_" + str(MAX_LEN) + ".bin"


if not os.path.exists(train_binary) :
    preprocess_dataset('TP_ISD2020/QUAERO_FrenchMed/MEDLINE/MEDLINEtrain_layer1_ID.csv', train_binary, kind='train')
if not os.path.exists(valid_binary) :
    preprocess_dataset('TP_ISD2020/QUAERO_FrenchMed/MEDLINE/MEDLINEdev_layer1_ID.csv', valid_binary, kind='valid')
if not os.path.exists(test_binary) :
    preprocess_dataset('TP_ISD2020/QUAERO_FrenchMed/MEDLINE/MEDLINEtest_layer1_ID.csv', test_binary, kind='test')



data_train = load_from_disk(train_binary)
data_test = load_from_disk(test_binary)
data_valid = load_from_disk(valid_binary)

Map:   0%|          | 0/11525 [00:00<?, ? examples/s]


ValueError: could not convert string to float: 'B-PROC'

In [63]:
trainer = Trainer(model=model, args=training_args, train_dataset=X_train, eval_dataset=y_train, tokenizer=tokenizer, compute_metrics=compute_metrics)
start = time.time()
trainer.train()
end = time.time()
elapsed = end - start

print("************** RESULTS for " + bert_model.replace("/","_") + " *****************")
print("Training time: " + str(elapsed) + " ms")
results_valid = trainer.evaluate(valid_loader)
print(results_valid)
results_test = trainer.evaluate(test_loader)
print(results_test)

C:\Users\Tyler Marino\AppData\Local\Temp\ipykernel_41712\2989772325.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, args=training_args, train_dataset=X_train, eval_dataset=y_train, tokenizer=tokenizer, compute_metrics=compute_metrics)
c:\Users\Tyler Marino\OneDrive\Desktop\GradSchool\Saclay\AI-Courses\TextMining-ChatBots\Venv_TextChat\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


AttributeError: 'list' object has no attribute 'keys'

In [16]:
# --- 7. FINAL TEST EVALUATION ---
print("\n--- TEST RESULTS ---")
model.load_state_dict(torch.load("best_cnn_ner.pt"))
model.eval()
test_true, test_pred = [], []

with torch.no_grad():
    for x, y in test_loader:
        x = x.to(DEVICE)
        out = model(x)
        preds = torch.argmax(out, dim=2).cpu().numpy()
        labels = y.numpy()
        for i in range(len(x)):
            p_s, t_s = [], []
            for j in range(SEQUENCE_LENGTH):
                if labels[i][j] == 0:
                    break
                p_s.append(idx2tag[preds[i][j]])
                t_s.append(idx2tag[labels[i][j]])
            test_pred.append(p_s)
            test_true.append(t_s)

print(classification_report(test_true, test_pred))


--- TEST RESULTS ---
              precision    recall  f1-score   support

        ANAT       0.38      0.24      0.30       364
        CHEM       0.45      0.19      0.27      1037
        DEVI       0.15      0.04      0.06       107
        DISO       0.35      0.24      0.29       977
        GEOG       0.50      0.05      0.09        63
        LIVB       0.75      0.57      0.65       498
        OBJC       0.14      0.04      0.06        81
        PHEN       0.00      0.00      0.00        70
        PHYS       0.48      0.21      0.29       190
        PROC       0.41      0.49      0.45       761

   micro avg       0.44      0.30      0.35      4148
   macro avg       0.36      0.21      0.24      4148
weighted avg       0.43      0.30      0.34      4148



/Users/idrissamahamoudoudicko/anaconda3/envs/HandonNLP/lib/python3.13/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### LSTM

In [17]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader
from gensim.models import Word2Vec
from seqeval.metrics import classification_report, f1_score
from tqdm import tqdm
import os

In [ ]:
# --- 2. CONFIGURATION ---
HIDDEN_DIM = 256
LSTM_EPOCHS = 30
LEARNING_RATE = 0.001

print(f"--- SWITCHING TO Bi-LSTM ---")
print(f"Vocab Size: {vocab_size}")
print(f"Embedding Dim: {EMBEDDING_DIM}")

--- SWITCHING TO Bi-LSTM ---
Vocab Size: 5775
Embedding Dim: 100


In [23]:
# --- 3. Bi-LSTM MODEL DEFINITION ---
class LSTM_NER(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes, pretrained_weights):
        super(LSTM_NER, self).__init__()

        # 1. Embedding Layer
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)

        # Initialize with TP1 Weights (Same as CNN)
        self.embedding.weight.data.copy_(pretrained_weights)
        # Optional: Unfreeze to allow fine-tuning
        self.embedding.weight.requires_grad = True

        # 2. Bi-LSTM Layer
        # batch_first=True -> Input is (Batch, Seq, Feature)
        # bidirectional=True -> Output is Hidden * 2
        self.lstm = nn.LSTM(embed_dim, hidden_dim,
                            batch_first=True, bidirectional=True)

        self.dropout = nn.Dropout(0.5)

        # 3. Fully Connected Layer
        # Maps from [Hidden * 2] to [Num_Tags]
        self.fc = nn.Linear(hidden_dim * 2, num_classes)

    def forward(self, x):
        # x shape: [Batch, Seq_Len]

        # Get Embeddings
        embeds = self.embedding(x)
        # embeds shape: [Batch, Seq_Len, Embed_Dim]

        # LSTM Pass
        # No permutation needed here (LSTM expects Batch, Seq, Dim)
        lstm_out, _ = self.lstm(embeds)
        # lstm_out shape: [Batch, Seq_Len, Hidden_Dim * 2]

        lstm_out = self.dropout(lstm_out)

        # Projection to Tag Space
        logits = self.fc(lstm_out)
        # logits shape: [Batch, Seq_Len, Num_Classes]

        return logits


# Initialize Model
model = LSTM_NER(vocab_size, EMBEDDING_DIM, HIDDEN_DIM,
                 len(tag2idx), embedding_weights).to(DEVICE)

In [24]:
weights = torch.ones(len(tag2idx)).to(DEVICE)
if 'O' in tag2idx:
    weights[tag2idx['O']] = 0.5

criterion = nn.CrossEntropyLoss(weight=weights, ignore_index=0)
optimizer = Adam(model.parameters(), lr=LEARNING_RATE)

In [30]:
print(f"\n--- STARTING Bi-LSTM TRAINING ON {DEVICE} ---")
best_f1 = 0

for epoch in range(LSTM_EPOCHS):
    model.train()
    train_loss = 0

    # Training Step
    for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        x, y = x.to(DEVICE), y.to(DEVICE)

        optimizer.zero_grad()
        out = model(x)  # [Batch, Seq, Num_Classes]

        # Flatten for Loss: (Batch * Seq, Num_Classes) vs (Batch * Seq)
        loss = criterion(out.view(-1, len(tag2idx)), y.view(-1))

        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # Validation Step
    model.eval()
    all_true, all_pred = [], []
    with torch.no_grad():
        for x, y in valid_loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            out = model(x)

            # Get Predictions
            preds = torch.argmax(out, dim=2).cpu().numpy()
            labels = y.cpu().numpy()

            # Decode (Remove Padding)
            for i in range(len(x)):
                p_s, t_s = [], []
                for j in range(SEQUENCE_LENGTH):
                    if labels[i][j] == 0:
                        break  # Stop at padding
                    p_s.append(idx2tag[preds[i][j]])
                    t_s.append(idx2tag[labels[i][j]])
                all_pred.append(p_s)
                all_true.append(t_s)

    # Metrics
    val_f1 = f1_score(all_true, all_pred)
    print(f"Loss: {train_loss/len(train_loader):.4f} | Val F1: {val_f1:.4f}")

    # Save Best
    if val_f1 > best_f1:
        best_f1 = val_f1
        torch.save(model.state_dict(), "best_lstm_ner.pt")


--- STARTING Bi-LSTM TRAINING ON cpu ---


NameError: name 'LSTM_EPOCHS' is not defined

In [26]:
print("\n--- Bi-LSTM TEST RESULTS ---")
model.load_state_dict(torch.load("best_lstm_ner.pt"))
model.eval()
test_true, test_pred = [], []

with torch.no_grad():
    for x, y in test_loader:
        x = x.to(DEVICE)
        out = model(x)
        preds = torch.argmax(out, dim=2).cpu().numpy()
        labels = y.numpy()

        for i in range(len(x)):
            p_s, t_s = [], []
            for j in range(SEQUENCE_LENGTH):
                if labels[i][j] == 0:
                    break
                p_s.append(idx2tag[preds[i][j]])
                t_s.append(idx2tag[labels[i][j]])
            test_pred.append(p_s)
            test_true.append(t_s)

print(classification_report(test_true, test_pred))


--- Bi-LSTM TEST RESULTS ---
              precision    recall  f1-score   support

        ANAT       0.34      0.13      0.19       364
        CHEM       0.26      0.22      0.24      1037
        DEVI       0.50      0.01      0.02       107
        DISO       0.41      0.28      0.33       977
        GEOG       0.00      0.00      0.00        63
        LIVB       0.75      0.52      0.62       498
        OBJC       0.00      0.00      0.00        81
        PHEN       0.00      0.00      0.00        70
        PHYS       0.38      0.16      0.22       190
        PROC       0.50      0.46      0.48       761

   micro avg       0.42      0.29      0.34      4148
   macro avg       0.31      0.18      0.21      4148
weighted avg       0.40      0.29      0.33      4148



### `French Press dataset`

In [27]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader
from gensim.models import Word2Vec
from seqeval.metrics import classification_report, f1_score
from tqdm import tqdm
import os
import gc

In [34]:
# --- 1. CONFIGURATION ---
SEQUENCE_LENGTH = 128
EMBEDDING_DIM = 100   # Must match your w2v_press_cbow.model
BATCH_SIZE = 32
EPOCHS = 15
LEARNING_RATE = 0.001
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# PATHS (Press Data & Press Embeddings)
W2V_PATH = "w2v_press_cbow.model"
PRESS_DIR = "TP_ISD2020/QUAERO_FrenchPress"

TRAIN_FILES = [f"{PRESS_DIR}/fra4_ID_train.csv"]
VALID_FILES = [f"{PRESS_DIR}/fra4_ID_dev.csv"]
TEST_FILES = [f"{PRESS_DIR}/fra4_ID_test.csv"]

In [35]:
def load_data_from_csv(file_paths):
    all_sentences, all_tags = [], []
    for fpath in file_paths:
        if not os.path.exists(fpath):
            continue
        print(f"Loading {os.path.basename(fpath)}...", end=" ")
        try:
            df = pd.read_csv(fpath, sep=None, engine="python",
                             keep_default_na=False, skip_blank_lines=False)
        except:
            continue

        if "Mot" in df.columns:
            words, tags = df["Mot"].astype(
                str).values, df["Tag"].astype(str).values
        else:
            words, tags = df.iloc[:, 0].astype(
                str).values, df.iloc[:, -1].astype(str).values

        curr_s, curr_t, file_s, file_t = [], [], [], []
        for w, t in zip(words, tags):
            if not w.strip():
                if curr_s:
                    file_s.append(curr_s)
                    file_t.append(curr_t)
                    curr_s, curr_t = [], []
            else:
                curr_s.append(w)
                curr_t.append(t)
        if curr_s:
            file_s.append(curr_s)
            file_t.append(curr_t)

        # Chunking
        if len(file_s) < 10 and len(words) > 500:
            flat_w = [w for s in file_s for w in s]
            flat_t = [t for s in file_t for t in s]
            file_s = [flat_w[i:i+SEQUENCE_LENGTH]
                      for i in range(0, len(flat_w), SEQUENCE_LENGTH)]
            file_t = [flat_t[i:i+SEQUENCE_LENGTH]
                      for i in range(0, len(flat_t), SEQUENCE_LENGTH)]

        print(f"-> {len(file_s)} sentences.")
        all_sentences.extend(file_s)
        all_tags.extend(file_t)
    return all_sentences, all_tags


print("--- LOADING PRESS DATA ---")
train_sents, train_tags = load_data_from_csv(TRAIN_FILES)
valid_sents, valid_tags = load_data_from_csv(VALID_FILES)
test_sents, test_tags = load_data_from_csv(TEST_FILES)


--- LOADING PRESS DATA ---
Loading fra4_ID_train.csv... -> 9034 sentences.
Loading fra4_ID_dev.csv... -> 744 sentences.
Loading fra4_ID_test.csv... -> 749 sentences.


In [36]:
# --- 3. PREPARE EMBEDDINGS (PRESS W2V) ---
print("\n--- LOADING PRESS EMBEDDINGS ---")
# Build Vocab
word_counts = {}
for sent in train_sents:
    for word in sent:
        word_counts[word] = word_counts.get(word, 0) + 1
vocab = sorted(word_counts, key=word_counts.get, reverse=True)
word2idx = {w: i+2 for i, w in enumerate(vocab)}
word2idx['<PAD>'] = 0
word2idx['<UNK>'] = 1
idx2word = {v: k for k, v in word2idx.items()}

# Load Weights
w2v_model = Word2Vec.load(W2V_PATH)
embedding_matrix = np.zeros((len(word2idx), EMBEDDING_DIM))
hits = 0
for word, idx in word2idx.items():
    if word in w2v_model.wv:
        embedding_matrix[idx] = w2v_model.wv[word]
        hits += 1
    elif word.lower() in w2v_model.wv:
        embedding_matrix[idx] = w2v_model.wv[word.lower()]
        hits += 1
    else:
        embedding_matrix[idx] = np.random.normal(
            scale=0.6, size=(EMBEDDING_DIM,))

print(f"Vocab: {len(word2idx)} | Coverage: {hits/len(word2idx):.1%}")
embedding_weights = torch.tensor(
    embedding_matrix, dtype=torch.float32).to(DEVICE)


--- LOADING PRESS EMBEDDINGS ---
Vocab: 37865 | Coverage: 99.5%


In [37]:
# --- 4. ENCODE DATA ---
def encode_seq(seqs, mapping, default):
    enc = []
    for s in seqs:
        r = [mapping.get(x, default) for x in s]
        if len(r) < SEQUENCE_LENGTH:
            r += [0]*(SEQUENCE_LENGTH-len(r))
        else:
            r = r[:SEQUENCE_LENGTH]
        enc.append(r)
    return torch.tensor(enc, dtype=torch.long)


X_train = encode_seq(train_sents, word2idx, 1)
X_valid = encode_seq(valid_sents, word2idx, 1)
X_test = encode_seq(test_sents,  word2idx, 1)

tag_set = set(t for s in train_tags+valid_tags+test_tags for t in s)
tag2idx = {t: i+1 for i, t in enumerate(sorted(list(tag_set)))}
tag2idx['<PAD>'] = 0
idx2tag = {v: k for k, v in tag2idx.items()}
print(f"Tags: {tag2idx}")

y_train = encode_seq(train_tags, tag2idx, 0)
y_valid = encode_seq(valid_tags, tag2idx, 0)
y_test = encode_seq(test_tags,  tag2idx, 0)

train_loader = DataLoader(TensorDataset(
    X_train, y_train), shuffle=True, batch_size=BATCH_SIZE)
valid_loader = DataLoader(TensorDataset(
    X_valid, y_valid), shuffle=False, batch_size=BATCH_SIZE)
test_loader = DataLoader(TensorDataset(X_test, y_test),
                         shuffle=False, batch_size=BATCH_SIZE)

Tags: {'O': 1, 'b-func': 2, 'b-loc': 3, 'b-org': 4, 'b-pers': 5, 'b-prod': 6, 'i-func': 7, 'i-loc': 8, 'i-org': 9, 'i-pers': 10, 'i-prod': 11, '<PAD>': 0}


### `Part A : CNN Model`

In [38]:
print("\n" + "="*30 + "\n TRAINING CNN (PRESS)\n" + "="*30)


class CNN_NER(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes, weights):
        super(CNN_NER, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.embedding.weight.data.copy_(weights)  # Init with Press W2V
        self.conv1 = nn.Conv1d(embed_dim, 128, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(128, 256, kernel_size=3, padding=1)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.embedding(x).permute(0, 2, 1)
        x = self.dropout(
            F.relu(self.conv2(self.dropout(F.relu(self.conv1(x))))))
        return self.fc(x.permute(0, 2, 1))


model_cnn = CNN_NER(len(word2idx), EMBEDDING_DIM, len(
    tag2idx), embedding_weights).to(DEVICE)
weights = torch.ones(len(tag2idx)).to(DEVICE)
if 'O' in tag2idx:
    weights[tag2idx['O']] = 0.5
opt = Adam(model_cnn.parameters(), lr=LEARNING_RATE)
crit = nn.CrossEntropyLoss(weight=weights, ignore_index=0)

for e in range(EPOCHS):
    model_cnn.train()
    for x, y in train_loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        opt.zero_grad()
        loss = crit(model_cnn(x).view(-1, len(tag2idx)), y.view(-1))
        loss.backward()
        opt.step()
    # Validation info
    print(f"Epoch {e+1} Complete", end="\r")

print("\n--- CNN EVALUATION ---")
model_cnn.eval()
test_true, test_pred = [], []
with torch.no_grad():
    for x, y in test_loader:
        out = model_cnn(x.to(DEVICE))
        preds = torch.argmax(out, dim=2).cpu().numpy()
        labels = y.numpy()
        for i in range(len(x)):
            p_s, t_s = [], []
            for j in range(SEQUENCE_LENGTH):
                if labels[i][j] == 0:
                    break
                p_s.append(idx2tag[preds[i][j]])
                t_s.append(idx2tag[labels[i][j]])
            test_pred.append(p_s)
            test_true.append(t_s)
print(classification_report(test_true, test_pred))


 TRAINING CNN (PRESS)
Epoch 15 Complete
--- CNN EVALUATION ---


/Users/idrissamahamoudoudicko/anaconda3/envs/HandonNLP/lib/python3.13/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: b-prod seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/Users/idrissamahamoudoudicko/anaconda3/envs/HandonNLP/lib/python3.13/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: i-prod seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/Users/idrissamahamoudoudicko/anaconda3/envs/HandonNLP/lib/python3.13/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: b-pers seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/Users/idrissamahamoudoudicko/anaconda3/envs/HandonNLP/lib/python3.13/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: i-pers seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/Users/idrissamahamoudoudicko/anaconda3/envs/HandonNLP/lib/python3.13/site-packa

              precision    recall  f1-score   support

        func       0.53      0.47      0.50       624
         loc       0.69      0.65      0.67       713
         org       0.24      0.45      0.31       505
        pers       0.79      0.63      0.70      1377
        prod       0.03      0.40      0.06       335

   micro avg       0.28      0.56      0.37      3554
   macro avg       0.46      0.52      0.45      3554
weighted avg       0.58      0.56      0.54      3554



### `PART B: LSTM MODEL`

In [39]:
print("\n" + "="*30 + "\n TRAINING LSTM (PRESS)\n" + "="*30)

# Cleanup Memory
del model_cnn, opt, crit
gc.collect()
torch.cuda.empty_cache() if torch.cuda.is_available() else None


class LSTM_NER(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes, weights):
        super(LSTM_NER, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.embedding.weight.data.copy_(weights)  # Init with Press W2V
        self.lstm = nn.LSTM(embed_dim, hidden_dim,
                            batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(hidden_dim*2, num_classes)

    def forward(self, x):
        embeds = self.embedding(x)
        lstm_out, _ = self.lstm(embeds)
        return self.fc(self.dropout(lstm_out))


model_lstm = LSTM_NER(len(word2idx), EMBEDDING_DIM, 256,
                      len(tag2idx), embedding_weights).to(DEVICE)
opt = Adam(model_lstm.parameters(), lr=LEARNING_RATE)
crit = nn.CrossEntropyLoss(weight=weights, ignore_index=0)

for e in range(EPOCHS):
    model_lstm.train()
    for x, y in train_loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        opt.zero_grad()
        loss = crit(model_lstm(x).view(-1, len(tag2idx)), y.view(-1))
        loss.backward()
        opt.step()
    print(f"Epoch {e+1} Complete", end="\r")

print("\n--- LSTM EVALUATION ---")
model_lstm.eval()
test_true, test_pred = [], []
with torch.no_grad():
    for x, y in test_loader:
        out = model_lstm(x.to(DEVICE))
        preds = torch.argmax(out, dim=2).cpu().numpy()
        labels = y.numpy()
        for i in range(len(x)):
            p_s, t_s = [], []
            for j in range(SEQUENCE_LENGTH):
                if labels[i][j] == 0:
                    break
                p_s.append(idx2tag[preds[i][j]])
                t_s.append(idx2tag[labels[i][j]])
            test_pred.append(p_s)
            test_true.append(t_s)
print(classification_report(test_true, test_pred))


 TRAINING LSTM (PRESS)
Epoch 15 Complete
--- LSTM EVALUATION ---
              precision    recall  f1-score   support

        func       0.15      0.46      0.23       624
         loc       0.69      0.65      0.66       713
         org       0.24      0.44      0.31       505
        pers       0.60      0.56      0.58      1377
        prod       0.05      0.32      0.09       335

   micro avg       0.27      0.52      0.35      3554
   macro avg       0.35      0.49      0.37      3554
weighted avg       0.44      0.52      0.45      3554

